<p><h3>1. Ler os dados</h3></p>
<span>Ler os dados  no arquivo csv, <b>Real_State_Properati</b></span>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import folium
from folium.plugins import HeatMap

df = pd.read_csv("Real_State_Properati.csv", sep = '\t')
df


<p><h3>2. Limpeza de dados</h3></p>
<span>Preparar os dados obtidos da leitura do arquivo csv</span>

<h4>2.1. Substituição de dados</h4>

In [ ]:
dfdiffsurfacezerado = df['surface_total_in_m2'] != 0 
df2 = df[dfdiffsurfacezerado]
dfGroup = df2.groupby(['property_type', 'place_name']).mean('surface_total_in_m2')
dfmerge = pd.merge(df, dfGroup, on=['property_type', 'place_name'])
dfmerge = dfmerge.rename(columns={'surface_total_in_m2_y': 'surface_total_in_m2_media'})
dfmerge['surface_total_in_m2_Final'] = np.where(dfmerge['surface_total_in_m2_x']== 0, dfmerge['surface_total_in_m2_media'] , dfmerge['surface_total_in_m2_x'])
dfmerge

In [ ]:
dfmerge = dfmerge.drop(columns={'surface_total_in_m2_x','lat_y','lon_y','price_y'})
dfmerge = dfmerge.rename(columns={'lat_x': 'lat'})
dfmerge = dfmerge.rename(columns={'lon_x': 'lon'})
dfmerge = dfmerge.rename(columns={'price_x': 'price'})
dfmerge

<h4>2.2. Conversão de dados</h4>

Adicionando uma coluna <b>price_usd</b> no dataframe, na qual valores que estão em peso argentino seram convertido para dolar americano

In [ ]:
dfmerge['price_usd']= dfmerge.apply(lambda x: x.price * 0.0085 if x.currency!='USD' else x.price,axis=1)
dfmerge

<h4>2.3. Criar informações baseadas nos dados</h4>

In [ ]:
dfmerge['preco_metro'] = dfmerge['price_usd'] / dfmerge['surface_total_in_m2_Final']
dfmerge

In [ ]:
############################## outlier MUITO ALTO
indexNames = dfmerge[ dfmerge['preco_metro'] > 206332 ].index
dfmerge.drop(indexNames , inplace=True)

<p><h3>3. Analisando os dados</h3></p>
<span>Demonstrar através de gráficos as informações obtidas dos dados do arquivo csv</span>

<h4>3.1. Preço do metro quadrado por bairro</h4>

In [ ]:

medias_bairro = dfmerge.groupby(['place_name'])['preco_metro'].agg(['max','min','mean','median'])
medias_bairro

<h5>3.1.1. Media do preço do metro quadrado por bairro</h5>

In [ ]:
medias_bairro = medias_bairro.sort_values(by=['mean'], ascending=True)
plt.figure(figsize = (45, 30))
plt.xlabel('Bairro', fontsize=14, color='red')
plt.ylabel('Preço do metro quadrado', fontsize=14, color='red')
plt.stem(medias_bairro.index, medias_bairro['mean'])
plt.xticks(rotation = 35)
plt.show()

<h5>3.1.2. Máximo do preço do metro quadrado por bairro</h5>

In [ ]:
medias_bairro = medias_bairro.sort_values(by=['max'], ascending=True)
plt.figure(figsize = (45, 30))
plt.xlabel('Bairro', fontsize=14, color='red')
plt.ylabel('Preço do metro quadrado', fontsize=14, color='red')
plt.stem(medias_bairro.index, medias_bairro['max'])
plt.xticks(rotation = 35)
plt.show()

<h5>3.1.3. Mínimo do preço do metro quadrado por bairro</h5>

In [ ]:
medias_bairro = medias_bairro.sort_values(by=['min'], ascending=True)
plt.figure(figsize = (45, 30))
plt.xlabel('Bairro', fontsize=14, color='red')
plt.ylabel('Preço do metro quadrado', fontsize=14, color='red')
plt.stem(medias_bairro.index, medias_bairro['min'])
plt.xticks(rotation = 35)
plt.show()

<h4>3.2. Preço do metro quadrado por tipo</h4>

In [ ]:
medias_tipos = dfmerge.groupby(['property_type'])['preco_metro'].agg(['max','min','mean','median'])
medias_tipos

<h5>3.2.1. Media do preço do metro quadrado por tipo</h5>

In [ ]:
array = dfmerge.to_numpy()

PH = []
apartment = []
house = []
store = []

for i in range(len(array.T[0])):
    if array.T[0][i] == 'PH':
        PH.append(array.T[11][i])
    if array.T[0][i] == 'apartment':
        apartment.append(array.T[11][i])
    if array.T[0][i] == 'house':
        house.append(array.T[11][i])
    if array.T[0][i] == 'store':
        store.append(array.T[11][i])

media_PH = np.mean(PH)
media_ap = np.mean(apartment)
media_houseH = np.mean(house)
media_store = np.mean(store)

x = ['PH', 'Apartament', 'House', 'Store']

y = [media_PH, media_ap, media_houseH, media_store]


plt.xlabel('Tipo do imovel', fontsize=14, color='red')
plt.ylabel('Preço do metro quadrado', fontsize=14, color='red')

plt.bar (x, y)

<h5>3.2.2. Máximo do preço do metro quadrado por tipo</h5>

In [ ]:
plt.figure(figsize = (15, 8))
plt.xlabel('Tipo do imovel', fontsize=14, color='red')
plt.ylabel('Preço do metro quadrado', fontsize=14, color='red')
plt.bar(medias_tipos.index, medias_tipos['max'])
plt.show()

<h5>3.2.3. Mínimo do preço do metro quadrado por tipo</h5>

In [ ]:
plt.figure(figsize = (15, 8))
plt.xlabel('Tipo do imovel', fontsize=14, color='red')
plt.ylabel('Preço do metro quadrado', fontsize=14, color='red')
plt.bar(medias_tipos.index, medias_tipos['min'])
plt.show()

<h4>3.3. Preço do metro quadrado por bairro e tipo</h4>

In [ ]:
medias_bairro_tipos = dfmerge.groupby(['property_type','place_name'])['preco_metro'].agg(['max','min','mean','median'])
medias_bairro_tipos

In [ ]:
medias_bairro_tipos_final = medias_bairro_tipos.reset_index([ 'property_type','place_name' ])
medias_bairro_tipos_final['place_name_type'] = medias_bairro_tipos_final[['property_type', 'place_name']].apply(' '.join, axis=1)
medias_bairro_tipos_final

<h5>3.3.1. Media do preço do metro quadrado por bairro e tipo</h5>

In [ ]:
plt.figure(figsize = (120, 60))
plt.xlabel('Bairro e tipos', fontsize=14, color='red')
plt.ylabel('Preço do metro quadrado', fontsize=14, color='red')
plt.stem(medias_bairro_tipos_final.place_name_type, medias_bairro_tipos_final['mean'])
plt.xticks(rotation = 35)
plt.show()

<h4>3.4. Mapa de calor</h4>

In [ ]:
map_hooray = folium.Map(location=[-34.62, -58.4],
                    zoom_start = 11) # Uses lat then lon. The bigger the zoom number, the closer in you get
map_hooray # Calls the map to display
#heat_df = dfmerge[dfmerge['Speed_limit']=='40'] # Reducing data size so it runs faster
#heat_df = dfmerge[dfmerge['Year']=='2007'] # Reducing data size so it runs faster
heat_df = dfmerge[['lat', 'lon']]
heat_df = heat_df.dropna(axis=0, subset=['lat','lon'])

# List comprehension to make out list of lists
heat_data = [[row['lat'],row['lon']] for index, row in heat_df.iterrows()]

# Plot it on the map
HeatMap(heat_data).add_to(map_hooray)

# Display the map
map_hooray

<p><h3>4. Os principais por preço do metro quadrado</h3></p>
<span>Demonstrar os cincos principais por valor do preço do metro quadrado</span>

<h4>4.1. Por localização do imovel</h4>

In [ ]:
medias_bairro_ordenada= medias_bairro.sort_values(by=['mean'], ascending=False)
medias_bairro_ordenada

<h5>4.1.1. Maior custo por metro quadrado</h5>

In [ ]:
medias_bairro_ordenada_5_maior=medias_bairro_ordenada.head(5)
medias_bairro_ordenada_5_maior

In [ ]:
plt.figure(figsize = (15, 8))
plt.xlabel('Bairro do imovel', fontsize=14, color='red')
plt.ylabel('Preço do metro quadrado', fontsize=14, color='red')
plt.barh(medias_bairro_ordenada_5_maior.index, medias_bairro_ordenada_5_maior['mean'], color='green')
plt.show()

<h5>4.1.2. Menor custo por metro quadrado</h5>

In [ ]:
medias_bairro_ordenada_5_menor=medias_bairro_ordenada.tail(5)
medias_bairro_ordenada_5_menor.sort_values(by=['mean'], ascending=True)

In [ ]:
plt.figure(figsize = (15, 8))
plt.xlabel('Bairro do imovel', fontsize=14, color='red')
plt.ylabel('Preço do metro quadrado', fontsize=14, color='red')
plt.barh(medias_bairro_ordenada_5_menor.index, medias_bairro_ordenada_5_menor['mean'],color='green')
plt.show()

<h4>4.2. Por localização e tipo do imovel</h4>

In [ ]:
medias_bairro_tipos_ordenada= medias_bairro_tipos_final.sort_values(by=['mean'], ascending=False)
medias_bairro_tipos_ordenada

<h5>4.2.1. Maior custo por metro quadrado</h5>

In [ ]:

medias_bairro_tipos_ordenada_5_maior=medias_bairro_tipos_ordenada.head(5)
medias_bairro_tipos_ordenada_5_maior


In [ ]:
plt.figure(figsize = (15, 8))
plt.xlabel('Bairro do imovel', fontsize=14, color='red')
plt.ylabel('Preço do metro quadrado', fontsize=14, color='red')
plt.scatter(medias_bairro_tipos_ordenada_5_maior.place_name_type, medias_bairro_tipos_ordenada_5_maior['mean'],color='red')
plt.show()

<h5>4.2.2. Menor custo por metro quadrado</h5>

In [ ]:
medias_bairro_tipos_ordenada_5_menor=medias_bairro_tipos_ordenada.tail(5)
medias_bairro_tipos_ordenada_5_menor.sort_values(by=['mean'], ascending=True)

In [ ]:
plt.figure(figsize = (15, 8))
plt.xlabel('Bairro do imovel', fontsize=14, color='red')
plt.ylabel('Preço do metro quadrado', fontsize=14, color='red')
plt.scatter(medias_bairro_tipos_ordenada_5_menor.place_name_type, medias_bairro_tipos_ordenada_5_menor['mean'],color='red')
plt.show()

<p><h3>5. Boxplot</h3></p>

In [ ]:
#criei uma tabela para cada tipo

df_PH = dfmerge[dfmerge['property_type'].isin(['PH'])]

df_apartment = dfmerge[dfmerge['property_type'].isin(['apartment'])]

df_house = dfmerge[dfmerge['property_type'].isin(['house'])]

df_store = dfmerge[dfmerge['property_type'].isin(['store'])]


In [ ]:
data_PH = df_PH['preco_metro'].tolist()
data_apartment = df_apartment['preco_metro'].tolist()
data_house = df_house['preco_metro'].tolist()
data_store = df_store['preco_metro'].tolist()

legenda = ['PH','Apartamento','House','Store']

data_boxplot = [data_PH, data_apartment, data_house, data_store]

fig = plt.figure(figsize =(10, 7))

ax = fig.add_axes([0, 0, 1, 1])
 
bp = ax.boxplot(data_boxplot,
               labels = legenda)

plt.show()

<h3>6. Macros Região</h3>

<h4>6.1. Metros quadrados totais por bairro</h4>

In [ ]:
soma_bairros = dfmerge.groupby(['place_name'])['surface_total_in_m2_Final'].agg(['sum'])
soma_bairros_ordenado = soma_bairros.sort_values(by=['sum'], ascending=False)


soma_bairros_ordenado

In [ ]:
soma_bairros_9 = soma_bairros_ordenado.head(9)
soma_bairros_9 = soma_bairros_9.reset_index(['place_name'])

soma_bairros_9

In [ ]:
todos = soma_bairros_ordenado[9:62].agg(['sum',])
todos


In [ ]:
soma_bairros_10 = soma_bairros_9.append(todos)


soma_bairros_final = soma_bairros_10.fillna('Outros')

soma_bairros_final

In [ ]:
# convert_dict = {'sum': int}

# soma_bairros_final = soma_bairros_final.astype(convert_dict)

soma_bairros_final = soma_bairros_10

soma_bairros_final.rename(columns = {'sum':'Soma'}, inplace = True)

soma_bairros_final = soma_bairros_10.fillna('Outros')

soma_bairros_final

In [ ]:

def func(pct, allvalues): 
    absolute = int(pct / 100.*np.sum(allvalues)) 
    return "{:.1f}%)".format(pct, absolute) 

fig, ax = plt.subplots(figsize =(15, 10.5)) 
wedges, texts, autotexts = ax.pie(soma_bairros_final.Soma, 
                                  autopct = lambda pct: func(pct, soma_bairros_final.Soma),
                                  #labels = soma_bairros_final.place_name)
                                 )
fig, ax.legend(wedges, soma_bairros_final.place_name,
              title = "Bairros",
              loc = "center right",
              bbox_to_anchor = (1, 0, 0.3, 1))
                                
plt.title('Total do metro quadrado por bairro.', color='red')

fig = plt.figure(figsize =(10, 7)) 



#plt.pie(soma_bairros_final.Soma, labels = soma_bairros_final.place_name)
 
#plt.show()9

<h4>6.2. Bairros com maior somatória de preço</h4>

In [ ]:
soma_preco = dfmerge.groupby(['place_name'])['price_usd'].agg(['sum'])
soma_preco_ordenado = soma_preco.sort_values(by=['sum'], ascending=False)

soma_preco_10 = soma_preco_ordenado.head(10)
soma_preco_10 = soma_preco_10.reset_index(['place_name'])

soma_preco_10

In [ ]:
soma_preco_10.rename(columns = {'sum':'Soma'}, inplace = True)
soma_preco_10

In [ ]:

def func(pct, allvalues): 
    absolute = int(pct / 100.*np.sum(allvalues)) 
    return "{:.1f}%)".format(pct, absolute) 
fig, ax = plt.subplots(figsize =(15, 10.5)) 
wedges, texts, autotexts = ax.pie(soma_preco_10.Soma, 
                                  autopct = lambda pct: func(pct, soma_preco_10.Soma),
                                  #labels = soma_preco_10.place_name
                                 )
fig, ax.legend(wedges, soma_bairros_final.place_name,
              title = "Bairros",
              loc = "center right",
              bbox_to_anchor = (1, 0, 0.3, 1))

plt.title('Total dos valores em dolar por bairro.', color='red')

fig = plt.figure(figsize =(10, 7)) 




#fig = plt.figure(figsize =(10, 7)) 
#plt.pie(soma_preco_10.Soma, labels = soma_preco_10.place_name)
#plt.show()

<h2>TRABALHO PRÁTICO II</h2>

<p><h3>7. Selecionando as informações</h3></p>
<span>Para o subconjunto, o tipo de propriedade escolhido foi o tipo "Apartment", pois é o unico tipo com registros suficientes para o trabalho. Os bairros ecolhidos foram Recoleta e Palermo, por serem bairros próximos um do outro e com grande quatidade de apartamentos.</span>

In [ ]:
dfApartament = dfmerge[(dfmerge["property_type"] == "apartment") & ((dfmerge["place_name"] == "Recoleta") | (dfmerge["place_name"] == "Palermo"))]
dfApartament

<h3>8. Regressão Linear</h3>

In [1029]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.inspection import permutation_importance
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

font = {'fontname':'Arial', 'size':'16', 'color':'black', 'weight':'normal'}

<h4>8.1. Regressão linear simples</h4>

In [ ]:
def info_simple(eixo_y, previsoes):
    #Avaliando desempemho
    print ("Avaliação de Desempenho:")    
    print ("MSE:", metrics.mean_squared_error(eixo_y, previsoes))
    print ("RMSE:",np.sqrt(metrics.mean_squared_error(eixo_y, previsoes)) )
    print ('MAE:', metrics.mean_absolute_error(eixo_y, previsoes))
    print ('R2:', metrics.r2_score(eixo_y, previsoes))
    

In [ ]:
def chart_simple(eixo_x, eixo_y, previsoes, label_x, label_y):
    
    plt.figure(figsize = (20,10))
    plt.scatter(eixo_x, eixo_y, color='blue' )
    plt.plot( eixo_x, previsoes)

    plt.title('Regressão linear simples', font)
    plt.xlabel(label_x, font)
    plt.ylabel(label_y, font)

In [ ]:

def generate_regression_simple(dadosApartment, label_x, label_y):
    dfAp = dfApartament[dadosApartment]
        
    eixo_x = dfAp.iloc[:,1].values 
    eixo_y = dfAp.iloc[:,0].values
    
    #Tranformando os dados em matriz
    eixo_x = eixo_x.reshape(-1, 1)
    eixo_y = eixo_y.reshape(-1, 1)

    funcao_Regressao = LinearRegression()
    model = funcao_Regressao.fit( eixo_x, eixo_y )
    previsoes = funcao_Regressao.predict(eixo_x)    
    
    
    chart_simple(eixo_x, eixo_y, previsoes, label_x, label_y)
    #Avaliando desempemho
    info_simple(eixo_y, previsoes)

<h5>8.1.1. Regressão por preço do metro quadrado</h5>

In [ ]:
dadosApartment_preco_metro = ["preco_metro",'surface_total_in_m2_Final']
generate_regression_simple(dadosApartment_preco_metro, 'Superficie total em m²', 'Preço por metro quadrado')

<h5>8.1.2. Regressão por preço em dólar</h5>

In [ ]:
dadosApartment_price_usd = ["price_usd",'surface_total_in_m2_Final']
generate_regression_simple(dadosApartment_price_usd, 'Superficie total em m²', 'Preço em dolar')

<h4>8.2. Regressão múltipla</h4>

In [ ]:
def info_multiple(model,X, y, predictions):
    print ('Intercepto = ', ' ', model.intercept_)
    print ('RM = ', ' ', model.coef_)
    print ('R2_train = ', ' ', model.score(X, y))
    
    print ("MSE:", metrics.mean_squared_error(y, predictions))
    print ("RMSE:",np.sqrt(metrics.mean_squared_error(y, predictions)) )
    print ('MAE:', metrics.mean_absolute_error(y, predictions))
    print ('R2:', metrics.r2_score(y, predictions))

In [ ]:
def chart_multiple(y,predictions, label_x, label_y):
        
    plt.figure(figsize = (20,10))    
    plt.plot(y, 
         y, 
         '-.', 
         c = 'grey'
        )
    
    plt.scatter(predictions, 
            y, 
            s = 50, 
            c = 'r', 
            marker = '+', 
            zorder = 10
           ) 
      
    plt.title('Regressão linear múltipla',font)
    plt.xlabel(label_x,font)
    plt.ylabel(label_y,font)
    plt.show()

<span>O método Permutation Importance será utilizado no modelo de Regressão Múltipla para avaliar o impacto das features sobre o modelo.</span>

In [ ]:
def generate_feature_importance(model, X, y):
    return permutation_importance(model, X, y, n_repeats=10, random_state=0)

In [ ]:
def info_feature_importance(model, X_test, y_test):
    feature_importance = generate_feature_importance(model, X_test, y_test)

    column_names = X_test.columns
    result = pd.Series(feature_importance.importances_mean, index=column_names)
    result.sort_values(ascending=False)
    print(result)
    print("------X-----X----X-------X---X---X")

In [ ]:
def generate_regression_multiple(variaveis, targets, label_x, label_y):
    X = dfApartament[variaveis]
    y = targets   
    
    # usa shuffle = False para evitar data leakage
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = False)
    linreg = LinearRegression()
    model_multiple = linreg.fit(X_train, y_train)
    y_pred = linreg.predict(X_test)
    info_feature_importance(model_multiple, X_test, y_test)
    info_multiple(model_multiple,X_test, y_test, y_pred)
    chart_multiple(y_test, y_pred, label_x, label_y)

In [ ]:
def generate_regression_multiple_normalizer(variaveis, targets, label_x, label_y):
    
    X = dfApartament[variaveis]
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(data_scaled, columns=X.columns)
    
    y = targets   
    
    # usa shuffle = False para evitar data leakage
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, shuffle = False)
    linreg = LinearRegression()
    model_multiple = linreg.fit(X_train, y_train)
    y_pred = linreg.predict(X_test)
    info_feature_importance(model_multiple, X_test, y_test)
    info_multiple(model_multiple,X_test, y_test, y_pred)
    chart_multiple(y_test, y_pred, label_x, label_y)

<h5>8.2.1. Regressão por preço do metro quadrado</h5>

In [ ]:
generate_regression_multiple(['lat', 'lon'], dfApartament["preco_metro"],"Previsões baseando na localização","Preço por metro quadrado")

In [ ]:
generate_regression_multiple(['lat', 'lon', 'price_usd'], dfApartament["preco_metro"],"Previsões baseando na localização e valor monetário","Preço por metro quadrado")

In [ ]:
generate_regression_multiple(['lat', 'lon', 'surface_total_in_m2_Final'], dfApartament["preco_metro"],"Previsões baseando na localização e área","Preço por metro quadrado")

In [ ]:
generate_regression_multiple(['lat', 'lon', 'surface_total_in_m2_Final','price_usd'], dfApartament["preco_metro"],"Previsões baseando na localização, área e valor monetário","Preço por metro quadrado")

Lembando que : Um valor negativo significa que estamos estimando pior do que usando a média.

<h5>8.2.2. Regressão por preço em dólar</h5>

In [ ]:
generate_regression_multiple(['lat', 'lon'], dfApartament["price_usd"],"Previsões baseando na localização","Preço em dólar")

In [ ]:
generate_regression_multiple(['lat', 'lon', 'preco_metro'], dfApartament["price_usd"],"Previsões baseando na localização e preço por metro quadrado","Preço em dólar")

In [ ]:
generate_regression_multiple(['lat', 'lon', 'surface_total_in_m2_Final'], dfApartament["price_usd"],"Previsões baseando na localização e área","Preço em dólar")

In [ ]:
generate_regression_multiple(['lat', 'lon', 'surface_total_in_m2_Final','preco_metro'], dfApartament["price_usd"],"Previsões baseando na localização, área e preço por metro quadrado","Preço em dólar")

<p><h5>8.2.3. Normalizando o modelo</h5></p>
Levando em consideração o objetivo final, no qual devemos focar no preço do metro quadrado, e baseando na <b>Permutation Importance</b>. Consideramos as variáveis que tem mais influência sobre o modelo e normalizamos os dados antes de treinar utilizando Standard Scaler.

In [ ]:
generate_regression_multiple_normalizer(['surface_total_in_m2_Final','price_usd'], dfApartament["preco_metro"],"Previsões baseando na área e valor monetário","Preço por metro quadrado")

<h4>8.3. Regressão RLM com statsmodels</h4>

<span>A regressão linear robusta tem maior eficiência para dados com muita discrepância, visto que, a estimativa usando os métodos de Huber são menos distorcidas por causa dos outlier.</span>

In [ ]:
def info_rlm(y, predictions_rlm, model):
    
    # Imprimimos o MSE e um resumo do modelo 
    print ("MSE:", metrics.mean_squared_error(y, predictions_rlm))
    print ("RMSE:",np.sqrt(metrics.mean_squared_error(y, predictions_rlm)) )
    print ('MAE:', metrics.mean_absolute_error(y, predictions_rlm))
    print ('R2:', metrics.r2_score(y, predictions_rlm))
    print (model.summary())

In [ ]:
def chart_rlm(y,predictions_rlm,label_x, label_y):
     # Plotamos os resultados
    plt.figure(figsize = (20,10))
    plt.plot(y, 
            y, 
            '-.', 
            c = 'grey'
            )

    plt.scatter(predictions_rlm, 
                y, 
                s = 30, 
                c = 'green', 
                marker = '+', 
                zorder = 10
            )
    plt.title('Regressão linear RLM', font)
    plt.xlabel(label_x, font)
    plt.ylabel(label_y, font)
    plt.show()





In [ ]:
def generate_regression_rlm(variaveis, targets,label_x, label_y):
    X = dfApartament[variaveis]
    y = dfApartament[targets]
    # Teremos que agregar explícitamente uma constante:
    X = sm.add_constant(X)
    resrlm = sm.RLM(y, X).fit()
    predictions_rlm = resrlm.predict(X)

    chart_rlm(y,predictions_rlm, label_x, label_y)
    info_rlm(y, predictions_rlm, resrlm)
    

<h5>8.3.1. Regressão por preço do metro quadrado</h5>

In [ ]:
generate_regression_rlm(['lat', 'lon', 'price_usd'],["preco_metro"],"Previsões baseando na localização e valor monetário", "Preço por metro quadrado")

In [ ]:
generate_regression_rlm(['lat', 'lon', 'surface_total_in_m2_Final', 'price_usd'],["preco_metro"],"Previsões baseando na localização, área e valor monetário", "Preço por metro quadrado")

In [ ]:
generate_regression_rlm(['lat', 'lon', 'surface_total_in_m2_Final'],["preco_metro"],"Previsões baseando na localização e área", "Preço por metro quadrado")

In [ ]:
generate_regression_rlm(['surface_total_in_m2_Final', 'price_usd'],["preco_metro"],"Previsões baseando na localização, área e valor monetário", "Preço por metro quadrado")

<h5>8.3.2. Regressão por preço em dólar</h5>

In [ ]:
generate_regression_rlm(['lat', 'lon', 'preco_metro'],["price_usd"],"Previsões baseando na localização e preço por metro quadrado", "Preço em dólar")

In [ ]:
generate_regression_rlm(['lat', 'lon', 'surface_total_in_m2_Final', 'preco_metro'],["price_usd"],"Previsões baseando na localização, área e preço por metro quadrado", "Preço em dólar")

In [ ]:
generate_regression_rlm(['lat', 'lon', 'surface_total_in_m2_Final'],["price_usd"],"Previsões baseando na localização e área", "Preço em dólar")

<h4>8.4. Regressão Decision Tree Regressor</h4>

<span>Apesar de não ser um modelo linear é uma otima implementação, pois outlier ou valores nulos não são prejudiciais. E também é um algoritmo que lida bem com dados nominais.</span>

In [ ]:
def chart_dtr(y, predictions, column_names):
    names = ', '.join(column_names.values)
    plt.figure(figsize = (20,10))
    plt.plot(y, 
         y, 
         '-.', 
         c = 'grey'
        )
    
    plt.scatter(predictions, 
            y, 
            s = 50, 
            c = 'purple', 
            marker = '+', 
            zorder = 10
           )   
    
    plt.title(f"Previsões baseando em {names}", font)
    plt.xlabel(f"Previsões", font)
    plt.ylabel("Preço por metro quatrado", font)
    plt.show()

In [ ]:
def generate_train_test(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = False)
    return X_train, X_test, y_train, y_test

In [ ]:
def info_dtr(tree_model,X_dtr_test, y_dtr_test, y_dtr_pred):
    print ('R2_train:', tree_model.score(X_dtr_test, y_dtr_test))
    print ("MSE:", metrics.mean_squared_error(y_dtr_test, y_dtr_pred))
    print ("RMSE:",np.sqrt(metrics.mean_squared_error(y_dtr_test, y_dtr_pred)) )
    print ('MAE:', metrics.mean_absolute_error(y_dtr_test, y_dtr_pred))
    print ('R2:', metrics.r2_score(y_dtr_test, y_dtr_pred))

In [ ]:
def generate_regression_dtr(variaveis):
    X_dtr = dfApartament[variaveis]
    scaler_dtr = StandardScaler()
    data_scaled_dtr = scaler_dtr.fit_transform(X_dtr)
    X_dtr_scaled = pd.DataFrame(data_scaled_dtr, columns=X_dtr.columns)

    y_dtr = dfApartament["preco_metro"]

    X_dtr_train, X_dtr_test, y_dtr_train, y_dtr_test = generate_train_test(X_dtr_scaled, y_dtr)

    tree_model = DecisionTreeRegressor(random_state=42)
    tree_model.fit(X_dtr_train, y_dtr_train)

    y_dtr_pred = tree_model.predict(X_dtr_test)

    info_dtr(tree_model,X_dtr_test, y_dtr_test, y_dtr_pred)
    chart_dtr(y_dtr_test, y_dtr_pred, X_dtr_test.columns)

In [ ]:
generate_regression_dtr(['lat', 'lon','surface_total_in_m2_Final','price_usd'])

In [ ]:
generate_regression_dtr(['surface_total_in_m2_Final','price_usd'])